In [3]:
from __future__ import unicode_literals
import pandas as pd

In [4]:
pd.options.display.max_columns = None

In [13]:
filepath = '~/Documents/Avito/'
item_info = pd.read_csv(filepath+'ItemInfo_train.csv', nrows=10, encoding='utf-8')
item_info.tail(3)
# item_pairs =pd.read_csv(filepath+'ItemPairs_train.csv', nrows=10, encoding='utf-8')
# item_pairs.rename(columns={'itemID_1':'itemID'}, inplace=True)
# df = pd.merge(item_info,item_pairs, on='itemID')

,itemID,categoryID,title,description,images_array,attrsJSON,price,locationID,metroID,lat,lon
7,15,32,Телевизор,"Телевизоры кинескопные, диагональ от 37 см. до...","11244051, 14467554, 2240467, 5099565, 8002433","{""Вид товара"":""Телевизоры и проекторы""}",1600.0,662810,NaN,57.622434,39.887894
8,16,27,Шуба мутоновая,"качественная, производство Казань, хорошее сос...","11762574, 316289, 4015142","{""Вид одежды"":""Женская одежда"", ""Предмет одежд...",1000.0,657600,NaN,56.495116,84.972128
9,19,88,Массажная накидка Beurer mg,Распродаем остатки. Все модели хиты продаж!!! ...,9722988,"{""Вид товара"":""Приборы и аксессуары""}",5000.0,637640,500701.0,55.640538,37.606065


In [6]:
item_infofull = pd.read_csv(filepath+'ItemInfo_train.csv', encoding='utf-8')
item_infofull.rename(columns={'itemID':'itemID_2'}, inplace=True)
df1 = pd.merge(df,item_infofull, on='itemID_2')

In [7]:
df1.head()
df2 = df1[['itemID', 'itemID_2', 'title_x', 'title_y']]
df2.apply(lambda x: pd.lib.infer_dtype(x.values))

/home/juan/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:3: FutureWarning: pandas.lib is deprecated and will be removed in a future version.
You can access infer_dtype as pandas.api.types.infer_dtype
  This is separate from the ipykernel package so we can avoid doing imports until


itemID      integer
itemID_2    integer
title_x     unicode
title_y     unicode
dtype: object

### Fuzzywuzzy 

AKA Approximate String Matching, finds strings that approximately matches a given pattern. 

Metric: **Levenshtein** (edit distance) : Number of primitive operations required to convert the string into an exact match

In [53]:
from fuzzywuzzy import fuzz

`token_set_ratio` is used as duplicated ads might contain titles that have duplicated words in a jumbled order

In [60]:
df2['fuzz_ratio'] = df2.apply(lambda x: fuzz.token_set_ratio(x['title_x'],x['title_y'])/100.0, axis=1)

/home/juan/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [61]:
df2['fuzz_ratio']

0    1.0
1    1.0
2    0.9
3    1.0
4    0.0
5    0.0
6    1.0
7    1.0
8    0.0
9    0.0
Name: fuzz_ratio, dtype: float64

### Jellyfish

In [22]:
import jellyfish

In [48]:
# Omit Hamming; undefined for strings with different length
df2['lev_dist'] = df2.apply(lambda x: jellyfish.levenshtein_distance(x['title_x'],x['title_y']), axis=1)                                             
df2['jaro_dist'] = df2.apply(lambda x: jellyfish.jaro_distance(x['title_x'], x['title_y']), axis=1)
df2['jarow_dist'] = df2.apply(lambda x: jellyfish.jaro_winkler(x['title_x'], x['title_y']), axis=1)

/home/juan/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/juan/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/juan/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

Se

,itemID,itemID_2,title_x,title_y,lev_dist,jaro_dist,jarow_dist
0,1,4112648,Продам Камаз 6520,Продам Камаз 6520,0,1.000000,1.000000
1,3,1991275,Yamaha r6,Yamaha R6,1,0.925926,0.955556
2,4,1223296,iPhone 3gs 8gb,iPhone 3gs 32gb,2,0.931746,0.959048
3,7,1058851,Xiaomi Mi4 3гб RAM + 16гб ROM белый,Смартфон Xiaomi Mi4 3гб RAM + 16гб ROM белый,9,0.788961,0.788961
4,8,2161930,Лыжные ботинки,Ботинки для лыж,13,0.360317,0.360317
